In [1]:
import numpy as np

import panda
import symmetry_utils as symm
import vector_space_utils as vs
import face_utils as fc
import quantum_utils as qm
import linprog as lp

# Vertices of LC
LC is the convex hull of LC1 and LC2. Let's first find the vertices of LC1.


### Vertices of LC1
We first generate a PANDA input file containing a facet description of LC1:

In [2]:
panda.write_panda_input_file(dimension=vs.dim_LC1((2,) * 7),
                             symmetries=symm.lc1_symm_generators(),
                             inequalities=-vs.construct_LC1_to_full_h()[:-1],
                             # minus sign because PANDA default is <= rather than >=. [:-1] because last row of homogeneous matrix is irrelevant.
                             filename='output/1_LC1_facets')

The file `1_LC1_facets` now can be loaded into PANDA to obtain a vertex description of LC1:
`panda -c -i safe -s reverse 1_LC1_facets`
This computation takes about a day and produces the contents of file `2_LC1_vertex_classes`, containing representatives of the 56 vertex classes of LC1 (see README.md for an explanation of the format).

### From LC1 to LC
The set of vertices of LC is the union of the vertices of LC1 and LC2. We have the vertex classes of LC1, but LC1 has some symmetries that LC doesn't (and vice versa). We therefore need to compute the equivalence classes of LC1 vertices under LC symmetries. Python is too slow for this, so we use a C++ program `panda-symmetries` that exploits some existing PANDA code. The code below creates a file that contains everything the C++ program needs: a representation of the LC1 symmetries and the LC symmetries, and the contents of `2_LC1_vertex_classes`, containing representatives of LC1-classes of LC1 vertices.

The resulting representatives will in fact be representatives of _all_ vertex classes of LC. This is because LC2 is the image of LC1 under exchanging the variables a1 with a2 and x1 with x2; but this is a symmetry of LC, so up to LC-equivalence we don't need to bother about LC2.

At one point in the code below we need to change coordinates from LC1 coordinates to NSS coordinates (see README.md). Here NSS is the no-(superluminal-)signalling set, which just like LC has dimension 86, rather than 80, the dimension of LC1.

In [3]:
# Symmetries:
# the first 7 symmetries will be LC1 symmetries (in NSS coordinates)
# the next 8 symmetries will be LC symmetries (in NSS coordinates)
maps = np.array([
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, b, (x1 + 1) % 2, x2, y)),  # x1 -> x1 + 1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: ((a1 + x1) % 2, a2, c, b, x1, x2, y)),  # a1 -> a1 + x1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, b, x1, (x2 + x1 * a1) % 2, y)), # x2 -> x2 + x1*a1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, (a2 + x1 * a1 * x2) % 2, c, b, x1, x2, y)),  # a2 -> a2 + x1*a1*x2
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, (c + x1 * a1 * x2 * a2) % 2, b, x1, x2, y)),  # c -> c + x1*a1*x2*a2
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, b, x1, x2, (y + 1) % 2)),  # y -> y  + 1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, (b + y) % 2, x1, x2, y)),  # b -> b  + y

    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, b, (x1 + 1) % 2, x2, y)),  # x1 -> x1 + 1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: ((a1 + x1) % 2, a2, c, b, x1, x2, y)),  # a1 -> a1 + x1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, b, x1, (x2 + 1) % 2, y)),  # x2 -> x2 + 1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, (a2 + x2) % 2, c, b, x1, x2, y)),  # a2 -> a2 + x2
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, (c + x1 * a1 * x2 * a2) % 2, b, x1, x2, y)),  # c -> c + x1*a1*x2*a2
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, b, x1, x2, (y + 1) % 2)),  # y  -> y  + 1
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a1, a2, c, (b + y) % 2, x1, x2, y)),  # b  -> b  + y
    symm.nss_var_perm_to_symm(lambda a1, a2, c, b, x1, x2, y: (a2, a1, c, b, x2, x1, y))  # a2 <-> a1, x2 <-> x1
])

# Load LC1 vertices
lc1_vertex_file = open('output/2_LC1_vertex_classes', 'r')
vertices_in_lc1_coords_h = []
for line in lc1_vertex_file.readlines():
    vertices_in_lc1_coords_h.append(list(map(int, line.split())))
vertices_in_lc1_coords_h = np.array(vertices_in_lc1_coords_h)

# Convert vertices to NSS coordinates
vertices_in_nss_coords_h = (vs.construct_full_to_NSS_h() @ vs.construct_LC1_to_full_h() @ vertices_in_lc1_coords_h.T).T

panda.write_panda_input_file(dimension=86,
                             symmetries=maps,
                             vertices=vertices_in_nss_coords_h,
                             filename='output/3_LC1_vertex_classes_to_unpack')

Running the resulting file `3_LC1_vertex_classes_to_unpack` through our C++ program,
`panda-symmetries --expand-and-reduce -i safe 3_LC1_vertex_classes_to_unpack > 4_LC_vertex_classes`
yields the 219 vertex classes of LC in `4_LC_vertex_classes`. See README.md for an explanation of the file format.

Finally, the file `5_all_lc_vertices.npy` is a numpy-dump of an integer array containing all vertices of LC (not just up to equivalence), obtained using the `--expand` option of the C++ program. It can be loaded as follows (takes up a bit of memory).

In [4]:
vertices = np.load('output/5_all_LC_vertices.npy')
print(vertices.shape)
del vertices  # to free up memory

(9165312, 87)


# Dimension of LC
With access to the vertices of LC, we can calculate its dimension, being one less than the size of a maximal set of affinely independent vertices. We'll use that a system of vectors is affinely independent if their homogeneous representations are linearly independent.

In [5]:
# It turns out the first 10000 vertices already contain 87 affinely independent ones
print("Dimension of LC:", np.linalg.matrix_rank(np.load('output/5_all_LC_vertices.npy')[:10000]) - 1)

Dimension of LC: 86


# Dimensions of faces
We can also use the vertices of LC to determine the dimensions of faces of LC defined by given inequalities.

In [6]:
ineq_i = fc.construct_ineq_nss(lambda a1, a2, c, b, x1, x2, y: (b == 0 and a2 == x1 and y == 0) * 1 / 4 +  # the 1/4 accounts for the probability p(x1,x2)
                                                               (b == 1 and a1 == x2 and y == 0) * 1 / 4 +
                                                               ((b + c) % 2 == x2 * y and x1 == 0) * 1 / 4, 7 / 4)
ineq_ii = fc.construct_ineq_nss(lambda a1, a2, c, b, x1, x2, y: (b == 0 and a2 == x1 and y == 0 and x2 == 0) * 1 / 2 +
                                                                (b == 1 and a1 == x2 and y == 0 and x1 == 0) * 1 / 2 +
                                                                ((b + c) % 2 == x2 * y and x1 == 0) * 1 / 4, 7 / 4)
ineq_iii = fc.construct_ineq_nss(lambda a1, a2, c, b, x1, x2, y: (b == 0 and a2 == x1 and y == 0 and x2 == 0) * 1 / 2 +
                                                                 (b == 1 and a1 == x2 and y == 0 and x1 == 1) * 1 / 2 +
                                                                 ((b + c) % 2 == x2 * y and x1 == 0) * 1 / 4 +
                                                                 (a2 == 1 and (c + 1) % 2 == b == y and x1 == x2 == 0) * 1 / 2, 7 / 4)
ineq_iv = fc.construct_ineq_nss(lambda a1, a2, c, b, x1, x2, y: ((a1 == x2 and a2 == x1) * 1 / 8), upper_bound=1 / 2)
ineq_v = fc.construct_ineq_nss(lambda a1, a2, c, b, x1, x2, y: ((a1 == x2 and a2 == x1 and b == 0 and y == 1) * 1 / 4 +
                                                                (b == 1 and y == 1) * 1 / 2 * 1 / 4), upper_bound=1 / 2)
ineq_vi = fc.construct_ineq_nss(lambda a1, a2, c, b, x1, x2, y: ((x1 * ((a1 + x2) % 2) == x2 * ((a2 + x1) % 2) == 0) * 1 / 8), upper_bound=3 / 4)
ineq_vii = fc.construct_ineq_nss(lambda a1, a2, c, b, x1, x2, y: ((x1 * ((a1 + x2) % 2) == x2 * ((a2 + x1) % 2) == 0 and b == 0 and y == 1) * 1 / 4 +
                                                                 (b == 1 and y == 1) * 3 / 4 * 1 / 4), upper_bound=3 / 4)

assert fc.max_violation_by_lc(ineq_i) == 0  # confirms that this inequality is valid and tight for LC (using linear programming)
assert fc.max_violation_by_lc(ineq_ii) == 0
assert fc.max_violation_by_lc(ineq_iii) == 0
assert fc.max_violation_by_lc(ineq_iv) == 0
assert fc.max_violation_by_lc(ineq_v) == 0
assert fc.max_violation_by_lc(ineq_vi) == 0
assert fc.max_violation_by_lc(ineq_vii) == 0
fc.count_face_dimension(ineq_i)   # -> 67
# fc.count_face_dimension(ineq_ii)   # -> 73
# fc.count_face_dimension(ineq_iii)   # -> 85
# fc.count_face_dimension(ineq_iv)   # -> 83
# fc.count_face_dimension(ineq_v)   # -> 85
# fc.count_face_dimension(ineq_vi)   # -> 83
# fc.count_face_dimension(ineq_vii)   # -> 85

## Finding facets
Due to the high number of vertices and dimensions of the LC polytope, it is unrealistic to enumerate its facets using PANDA. The function below can however be used to find facets adjacent to known faces (the higher the dimension of the known face, the shorter the computation). `ineq_iii`, `ineq_v` and `ineq_vii` above were found in similar ways.

In [10]:
# facet = fc.find_a_facet_adjacent_to_face(ineq_iv, np.load('output/5_all_LC_vertices.npy'), tol=1e-14)
# print(facet)
# assert fc.count_face_dimension(facet) == 85

The inequality supports a face of dimension 85


# Quantum correlations
`quantum_utils.py` provides functions to compute quantum correlations observed by parties using given quantum instruments in the four-partite process matrix consisting of a switch and an entangled observer. Some of these correlations violate the inequalities defined above.

In [8]:
qm_cor = qm.quantum_cor_nss_h_discardT(# Prepare T in state 0; CB in state Phi+
                                      rho_ctb=qm.rho_tcb_0phi,  
                                      # Alice 1 does a measure-prepare channel:
                                      instrs_A1=[qm.instr_measure_and_prepare(qm.z_onb, qm.ket0), qm.instr_measure_and_prepare(qm.z_onb, qm.ket1)],
                                      # Alice 2 does a measure-prepare channel:
                                      instrs_A2=[qm.instr_measure_and_prepare(qm.z_onb, qm.ket0), qm.instr_measure_and_prepare(qm.z_onb, qm.ket1)],
                                      # Charlie does a von Neumann projective mmt in the diagonal Z+X direction
                                      instr_C=qm.instr_vn_destr(qm.diag1_onb),
                                      # Bob chooses between Z and X:
                                      instrs_B=[qm.instr_vn_destr(qm.z_onb), qm.instr_vn_destr(qm.x_onb)])  
print("Inequality i is violated by", ineq_i @ qm_cor / qm_cor[-1])

Inequality i is violated by 0.015165042944955354


In addition, we can use linear programming to efficiently decide whether a given (quantum) correlation is a member of the LC polytope.

In [9]:
print(lp.is_cor_in_lc(qm_cor).success)

False
